Question 2A

##### When I accessed the site, and clicked in the women's handbag category, I picked up that every time I scroll down, it shows in the network console that I sent the below request:
##### 'https://www.michaelkors.com/on/demandware.store/Sites-mk_us-Site/en_US/Search-UpdateGrid?cgid=womens-handbags&start=0&sz=24'
##### here I identified that everytime i scroll down, value for 'sz' will increase. Hence I used this as a link for my request in the scraper. and to acquire all products at once, I started by changing the value for 'sz' to '500'
##### 'https://www.michaelkors.com/on/demandware.store/Sites-mk_us-Site/en_US/Search-UpdateGrid?cgid=womens-handbags&start=0&sz=500'


![alt text](image.png)

##### the site would look like this
![alt text](image-1.png)

design diagram: 
![alt text](design_diagram.png)

In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime
import time
import csv
import schedule

##### 1. Finding product tile wrapper
![alt text](image-2.png)

##### 2. Finding the product name
![alt text](image-3.png)

##### 3. Finding the price
![alt text](image-4.png)

##### 4. Finding a discounted price
![alt text](image-5.png)

##### 5. Finding the rating
![alt text](image-6.png)

In [ ]:
def scrape_michael_kors():

    #since the request could not be executed without headers, I added the headers to the request
    #The headers were copied from the network console when the inspect panel was opened 
    headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en",
    "cache-control": "max-age=0",
    "priority": "u=0, i",
    "sec-ch-ua": "\"Chromium\";v=\"130\", \"Google Chrome\";v=\"130\", \"Not?A_Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "Windows",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
    }
    url = 'https://www.michaelkors.com/on/demandware.store/Sites-mk_us-Site/en_US/Search-UpdateGrid?cgid=womens-handbags&start=0&sz=500'
    response = requests.get(url,headers=headers)


    #send a request to the URL
    soup = BeautifulSoup(response.content, 'html.parser')

    #see markdown after this section for screenshots and description of how the relevant sections were identified in the html content

    #parse the HTML content
    products = soup.find_all('div', {'class': 'product-tile-wrapper'})

    #Extract product name, price, discounted price ratings, timestamp
    data = []
    for product in products:
        name = product.find('a', class_='link back-to-product-anchor-js').text.strip()

    #For some products, there are original prices (if there is a separate discount). These may not appear in all products

        price = ""  # Default value in case price is not found
        list_span = product.find('span', {'class': 'list'})
        if list_span:
            value_span = list_span.find('span', {'class': 'value'})
            if value_span:
                price = value_span.text.strip()

    #For most products, there is a sales price
        sales_price = "No sales price"  # Default value in case price is not found

        sales_span = product.find('span', {'class': 'sales'})
        if sales_span:
            value_span_1 = sales_span.find('span', {'class': 'value'})
            if value_span_1:
                sales_price = value_span_1.text.strip()    

    #Not all products have ratings

        ratings = "" 

        ratings_span = product.find('div', {'class': 'ratings'})
        if ratings_span:
            value_span_2 = ratings_span.find('span', {'class': 'sr-only'})
            if value_span_2:
                ratings = value_span_2.text.strip()


        timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        data.append([name, price, sales_price, ratings, timestamp])

    #remove 'out of 5 Customer Rating' text from the ratings column
    for row in data:
        row[3] = row[3].replace(' out of 5 Customer Rating', '')

    #add column headers
    data.insert(0, ['Product Name', 'Original Price', 'Sales Price', 'Ratings', 'Timestamp'])

    #Generate current date
    current_date = datetime.datetime.now().strftime('%Y-%m-%d')
    file_name = f'michael_kors_data_{current_date}.csv'

    #write data to a CSV file
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        for row in data:
            writer.writerow(row)

#schedule the scraper to run at 9 am every day
schedule.every().day.at("09:00").do(scrape_michael_kors)

while True:
    schedule.run_pending()
    time.sleep(1)



# Question 2b

### Additional data points available in the html

##### 1. Product tags such as "Kors loves" which outlines the shop's recommendations. Capturing these data provides an additional variable for analysis when understanding price, ratings of products
![alt text](image-9.png)

##### 2. Product subcategories such as colours 
![alt text](image-11.png)

##### 3. images of products: Images, if made available, can be useful in downstream visualisation tools. Other technologies such as image recognition and computer vision can be applied. 
![alt text](image-12.png)

##### 4. Product reviews: For each product there are reviews with different data points available for analysis. These data are available on the sites html - which will enable deeper analysis such as understanding how the product appeals to different customer segments, how the product stand out and compare against each other when measured by different features.

###### 4.1 Review text data is available
![alt text](image-14.png)

###### 4.2 Customer data including gender and age
![alt text](image-15.png)

###### 4.3 Customer rating data across different features - such as quality, appearance
![alt text](image-16.png)

### Some data points that are not available on the html but potentially on the api end point

##### 1. Inventory data: For example text that indicates number of items sold this week is an important data point indicating popularity of the item
![alt text](image-10.png)

##### 2. User data: data related to behaviour on the website such as how many clicks there are for each item, add-to-cart clicks, check out clicks, are also important data that will enable analysis on each individual product, and also the effectiveness of the check out process to customers

##### 3. Customer data: Customers' order history, demographics, purchase frequency are data that help understanding how different product types appeal to different segments (across geographies, age) and lifecycle of a product (especially with purchase history and frequency data). These data are probably not available for scraping and would only be accessible through secured or authenticated API end points. 

